In [ ]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier, DummyRegressor
from proxy_models import TargetMeanClassifier, TargetMeanRegressor
from utils import make_cv_function
from sklearn.pipeline import Pipeline
import openml
import pandas as pd
from sympy import Dummy, rem
from utils import get_benchmark_dataIDs, get_metadata_df
from ft_detection import FeatureTypeDetector



### Code for in-depth duplicate investigation

In [ ]:
import openml
import pandas as pd
from sympy import Dummy, rem
from utils import get_benchmark_dataIDs, get_metadata_df
from ft_detection import FeatureTypeDetector

benchmark = "TabArena"  # or "TabArena", "TabZilla", "Grinsztajn"
dataset_name = 'APS'  # 'Amazon_employee_access', 'nyc-taxi-green-dec-2016', 'adult', 'bank-marketing', 'credit-g', 'diabetes', 'german-credit', 'heart-disease', 'house-votes-84', 'iris', 'mushroom', 'wine-quality-red'

tids, dids = get_benchmark_dataIDs(benchmark)  

remaining_cols = {}

batch_sizes = {}

for tid, did in zip(tids, dids):
    task = openml.tasks.get_task(tid)  # to check if the datasets are available
    data = openml.datasets.get_dataset(did)  # to check if the datasets are available
    # if dataset_name not in data.name:
    #     continue
    # print(f"Processing dataset: {data.name} (ID: {tid})")
    # else:
    #     break
    print(data.name)
    X, _, _, _ = data.get_data()
    y = X[data.default_target_attribute]
    X = X.drop(columns=[data.default_target_attribute])

    if X.shape[0] < 5000 and X.shape[1] < 1000:
        batch_sizes[data.name] = 128
    elif X.shape[0] < 10000 and X.shape[1] > 1000:
        batch_sizes[data.name] = 16
    elif X.shape[0] > 10000 and X.shape[1] < 1000:
        batch_sizes[data.name] = 16
    else:
        batch_sizes[data.name] = 32

    
#     if benchmark == "Grinsztajn" and X.shape[0]>10000:
#         X = X.sample(10000, random_state=0)
#         y = y.loc[X.index]

#     if task.task_type == "Supervised Classification":
#         target_type = "binary" if y.nunique() == 2 else "multiclass"
#     else:
#         target_type = 'regression'
#     if target_type=="multiclass":
#         # TODO: Fix this hack
#         y = (y==y.value_counts().index[0]).astype(int)  # make it binary
#         target_type = "binary"
#     elif target_type=="binary" and y.dtype not in ["int", "float", "bool"]:
#         y = (y==y.value_counts().index[0]).astype(int)  # make it numeric
#     else:
#         y = y.astype(float)
#     print(data.name)
#     print(pd.Series({r: pd.concat([X.round(r).map(lambda x: 0.0 if x == 0.0 else x),y], axis=1).duplicated().mean() for r in [0,1,2,3,4,5,6,7,8,9,10]}).sort_values(ascending=False))
#     print("--"*20)

#     cv_func = make_cv_function(target_type=target_type)
#     target_model = TargetMeanClassifier if target_type == "binary" else TargetMeanRegressor
#     dummy_model = DummyClassifier(strategy='prior') if target_type == "binary" else DummyRegressor(strategy='mean')
#     print(data.name)
#     print(f"DUMMY: {np.mean(cv_func(X, y, Pipeline([('model', dummy_model)])))}")
#     print(f"DUP-PRED: {np.mean(cv_func(X.astype(str).sum(axis=1), y, Pipeline([('model', target_model())])))}")
#     print(f"DUP-PRED-ROUND0: {np.mean(cv_func(X.round(0).astype(str).sum(axis=1), y, Pipeline([('model', target_model())])))}")
#     print(f"DUP-PRED-ROUND1: {np.mean(cv_func(X.round(1).astype(str).sum(axis=1), y, Pipeline([('model', target_model())])))}")
#     print(f"DUP-PRED-ROUND2: {np.mean(cv_func(X.round(2).astype(str).sum(axis=1), y, Pipeline([('model', target_model())])))}")
#     print(f"DUP-PRED-ROUND3: {np.mean(cv_func(X.round(3).astype(str).sum(axis=1), y, Pipeline([('model', target_model())])))}")
#     print(f"SUM: {np.mean(cv_func(X.sum(axis=1), y, Pipeline([('model', target_model())])))}")
#     print(f"PROD: {np.mean(cv_func(X.prod(axis=1), y, Pipeline([('model', target_model())])))}")
#     print("--"*20)



# # anneal: 0=
# # diamonds: 0=0.07
# # hazelnut: 3: 0.22, 2: 0.99
# # maternal_health_risk: 0=0.64
# # physiochemical_protein: 0=0.056
# # qsar-biodeg: 0=0.05
# # wine_quality: slightly increases

In [ ]:
for col in X.columns:
    a = X[col].round(3).value_counts()
    b = X[col].round(3).map(lambda x: 0.0 if x == 0.0 else x).astype(str).value_counts()
    print(f"{col}: {a.shape} vs {b.shape}")
    if a.shape!=b.shape:
        break

In [ ]:
pd.crosstab(X.round(2).map(lambda x: 0.0 if x == 0.0 else x).astype(str).sum(axis=1),y).sort_values(1)

In [ ]:
for r in range(6):
    print(f"{r}: {X.round(r).map(lambda x: 0.0 if x == 0.0 else x).astype(str).duplicated().mean():3f},{pd.concat([X.round(r).map(lambda x: 0.0 if x == 0.0 else x).astype(str),y], axis=1).duplicated().mean():3f}")
    print(f"{r}: {X.round(r).duplicated().mean()}, {pd.concat([X.round(r),y], axis=1).duplicated().mean()}")
    print("--"*20)

In [ ]:
from sklearn.pipeline import Pipeline
from proxy_models import TargetMeanClassifier, TargetMeanRegressor
from utils import make_cv_function
cv_func = make_cv_function(target_type=target_type)
print(np.mean(cv_func(X.round(2).astype(str).sum(axis=1), y, Pipeline([('model', TargetMeanClassifier())]))))

# .82 if we round to 2 decimals
# .85 if we round to 3 decimals
X_curr = X.copy()
for iter in range(X.shape[1]):
    res = pd.Series({col: np.mean(cv_func(X_curr.drop(col,axis=1).round(3).astype(str).sum(axis=1), y, Pipeline([('model', TargetMeanClassifier())]))) for col in X_curr.columns})
    best = res.idxmax()
    print(best + f": {res.max()}")
    X_curr = X_curr.drop(best, axis=1)


In [ ]:
from itertools import product, combinations

X_curr = X.copy()
col_set = []
eligible_cols = set(X_curr.columns.tolist())

for iter in range(X.shape[1]):
    combs = list(product(eligible_cols, [0,1,2,3,4,5]))
    res = pd.Series({col+"__"+f"{r}": np.mean(cv_func(pd.concat([X_curr[col_set],X_curr[col].round(r)],axis=1).astype(str).sum(axis=1), y, Pipeline([('model', TargetMeanClassifier())]))) for col,r in combs})
    best = res.idxmax()
    print(best + f": {res.max()}")
    col_set.append(best)
    eligible_cols.remove(best.split("__")[0])  # remove the original column from eligible columns
    X_curr[best.split("__")[0]] = X_curr[best.split("__")[0]].round(int(best.split("__")[1]))
    

### Code for checking numerical precision

In [ ]:
import openml
import pandas as pd
from sympy import rem
from utils import get_benchmark_dataIDs, get_metadata_df
from ft_detection import FeatureTypeDetector

benchmark = "TabArena"  # or "TabArena", "TabZilla", "Grinsztajn"
dataset_name = 'superconductivity'  # 'Amazon_employee_access', 'nyc-taxi-green-dec-2016', 'adult', 'bank-marketing', 'credit-g', 'diabetes', 'german-credit', 'heart-disease', 'house-votes-84', 'iris', 'mushroom', 'wine-quality-red'

tids, dids = get_benchmark_dataIDs(benchmark)  

remaining_cols = {}

for tid, did in zip(tids, dids):
    task = openml.tasks.get_task(tid)  # to check if the datasets are available
    data = openml.datasets.get_dataset(did)  # to check if the datasets are available
    # if data.name!=dataset_name:
    #     continue
    # else:
    #     break
    print(data.name)
    X, _, _, _ = data.get_data()
    y = X[data.default_target_attribute]
    X = X.drop(columns=[data.default_target_attribute])
    
    if benchmark == "Grinsztajn" and X.shape[0]>10000:
        X = X.sample(10000, random_state=0)
        y = y.loc[X.index]

    if task.task_type == "Supervised Classification":
        target_type = "binary" if y.nunique() == 2 else "multiclass"
    else:
        target_type = 'regression'
    if target_type=="multiclass":
        # TODO: Fix this hack
        y = (y==y.value_counts().index[0]).astype(int)  # make it binary
        target_type = "binary"
    elif target_type=="binary" and y.dtype not in ["int", "float", "bool"]:
        y = (y==y.value_counts().index[0]).astype(int)  # make it numeric
    else:
        y = y.astype(float)

    X_num = X.select_dtypes(include=[float])
    X_num = X_num.loc[:,~(X_num.fillna(-9999999)==X_num.fillna(-9999999).astype(int)).all().values]
    print(f"FP-Numerical features in {data.name}: {X_num.shape[1]}/{X.shape[1]}")
    if X_num.shape[1] == 0:
        print(f"No numerical features in {data.name}. Skipping...")
        continue

    # IDEA for numerical resolution detection: Round the values to the value of highest difference between to neighbouring values
    from decimal import Decimal

    # def _precision(x):
    #     # turn into a Decimal so scientific notation is handled,
    #     # then look at the negative exponent
    #     d = Decimal(str(x)).normalize()
    #     return max(-d.as_tuple().exponent, 0)

    # precisions = {}
    # for col in X_num.select_dtypes(include=['float']).columns:
    #     nonzero = X_num[col].dropna().loc[lambda s: s != 0]
    #     if nonzero.empty:
    #         precisions[col] = 0
    #     else:
    #         precisions[col] = nonzero.map(_precision).max()

    # current_precision = pd.Series(precisions, dtype=int)

    def count_decimals_np(x, ndigits=2):
        # format with at most ndigits, no sci‐notation, no trailing zeros
        s = np.format_float_positional(x, precision=ndigits, trim='-')
        # ensure there’s a decimal point
        if "." not in s:
            return 0
        return len(s.split(".", 1)[1])

    precisions = {}
    for col in X_num.select_dtypes("float").columns:
        nonzero = X_num[col].dropna().loc[lambda s: s != 0]
        if nonzero.empty:
            precisions[col] = 0
        else:
            precisions[col] = nonzero.map(lambda x: count_decimals_np(x, ndigits=10)).max()

    current_precision = pd.Series(precisions, dtype=int)

    min_diff = pd.Series({col: pd.Series(X_num[col].dropna().unique()).sort_values().diff().min() for col in X_num.columns if X_num[col].dtype in [float]})
    min_diff_precision = min_diff.map(lambda x: count_decimals_np(x, ndigits=10))
    # min_diff_precision = min_diff.map(_precision)
    # min_diff_precision = min_diff.apply(lambda x: len(str(x).split('.')[1]))

    red = pd.concat([min_diff_precision, current_precision], axis=1).diff(axis=1)[1].min()
    red_col = pd.concat([min_diff_precision, current_precision], axis=1).diff(axis=1)[1].idxmin()

    print(f'Highest reduction: {red} in {red_col}')

    if data.name == 'superconductivity':
        break


In [ ]:

def count_decimals_np(x, ndigits=2):
    # format with at most ndigits, no sci‐notation, no trailing zeros
    s = np.format_float_positional(x, precision=ndigits, trim='-')
    # ensure there’s a decimal point
    if "." not in s:
        return 0
    return len(s.split(".", 1)[1])

precisions = {}
for col in X_num.select_dtypes("float").columns:
    nonzero = X_num[col].dropna().loc[lambda s: s != 0]
    if nonzero.empty:
        precisions[col] = 0
    else:
        precisions[col] = nonzero.map(lambda x: count_decimals_np(x, ndigits=2)).max()

current_precision = pd.Series(precisions, dtype=int)
current_precision

In [ ]:
def round_up_np(arr, decimals=0):
    factor = 10.0 ** decimals
    return np.ceil(arr * factor) / factor
# IDEA for numerical resolution detection: Round the values to the value of highest difference between to neighbouring values
max_diff = pd.Series({col: X[col].sort_values().diff().max() for col in X.columns}).sort_values(ascending=False)
round_up_np(max_diff)

# IDEA for numerical resolution detection: Round the values to the value of highest difference between to neighbouring values
current_precision = pd.Series({col: int(X[col].apply(lambda x: len(str(x).split('.')[1].rstrip('0'))).max()) for col in X.columns if X[col].dtype in [float]})

min_diff = pd.Series({col: pd.Series(X[col].unique()).sort_values().diff().min() for col in X.columns if X[col].dtype in [float]})
min_diff_precision = min_diff.apply(lambda x: len(str(x).split('.')[1].rstrip('0')))

red = pd.concat([min_diff_precision, current_precision], axis=1).diff(axis=1)[1].min()
red_col = pd.concat([min_diff_precision, current_precision], axis=1).diff(axis=1)[1].idxmin()

print(f'Highest reduction: {red} in {red_col}')

### Combining all proxy models and learning from them

In [ ]:
from category_encoders import LeaveOneOutEncoder, TargetEncoder
from sklearn.metrics import roc_auc_score
from utils import *
for col in [col]:
    nunique = X[col].nunique()
    x = X[col]
    print(f"Column: {col} with {X[col].nunique()} unique values")
    if target_type == 'binary':
        metric = roc_auc_score
        print('Linear: ', metric(y, UnivariateLogisticClassifier().fit(X[col].astype(float).fillna(0).to_frame(), y).predict_proba(X[col].astype(float).fillna(0).to_frame())[:,1]))
        print('Poly-2: ', metric(y, PolynomialLogisticClassifier(degree=2).fit(X[col].astype(float).fillna(0).to_frame(), y).predict_proba(X[col].astype(float).fillna(0).to_frame())[:,1]))
        print('Poly-3: ', metric(y, PolynomialLogisticClassifier(degree=3).fit(X[col].astype(float).fillna(0).to_frame(), y).predict_proba(X[col].astype(float).fillna(0).to_frame())[:,1]))
    else:
        metric = root_mean_squared_error
        print('Linear: ', metric(y, UnivariateLinearRegressor().fit(X[col].astype(float).fillna(0).to_frame(), y).predict(X[col].astype(float).fillna(0).to_frame())))
        print('Poly-2: ', metric(y, PolynomialRegressor(degree=2).fit(X[col].astype(float).fillna(0).to_frame(), y).predict(X[col].astype(float).fillna(0).to_frame())))
        print('Poly-3: ', metric(y, PolynomialRegressor(degree=3).fit(X[col].astype(float).fillna(0).to_frame(), y).predict(X[col].astype(float).fillna(0).to_frame())))
    print('LGB Binning')
    for m_bin in [2,4,8,12,16,32,64,128,256,512,1024,2048,min([nunique-100,4096])]:
        if m_bin <= nunique and m_bin > 1:
            pipe = Pipeline([
                ('binner', LightGBMBinner(max_bin=m_bin)),
                ('model', TargetMeanClassifier()) if target_type == 'binary' else ('model', TargetMeanRegressor())
            ])
            print(f"Bins: {m_bin}, Score: {np.mean(detector.cv_scores_with_early_stopping(X[col].to_frame(), y, pipe))}")
    
    print('Kmeans Binning')
    for m_bin in [2,4,8,12,16,32,64,128,256,512,1024,2048,min([nunique-100,4096])]:
        if m_bin <= nunique and m_bin > 1:
            pipe = Pipeline([
                ('KMeans-binner', KMeansBinner(max_bin=m_bin)),
                ('model', TargetMeanClassifier()) if target_type == 'binary' else ('model', TargetMeanRegressor())
            ])
            print(f"Bins: {m_bin}, Score: {np.mean(detector.cv_scores_with_early_stopping(X[col].to_frame(), y, pipe))}")

    print('Leave-One-Out: ', metric(y, LeaveOneOutEncoder().fit_transform(X[col].astype('category'), y)[col]))
    print('Target Encoding: ', metric(y, TargetEncoder(min_samples_leaf=1, smoothing=1e-10).fit_transform(X[col].astype('category'), y)[col]))
    
    print('--'*50)

### Finding a good threshold for categorical features

In [ ]:
stats = {}
for col in X.columns:
    stats[col] = get_optimal_cat_thresh(X[col], y, target_type='regression', verbose=True)


### Notes on cat detection

In [ ]:
# Still mistakes: 
# concrete_compressive_strength: Almost all are detected as categorical, but they are numerical
# credit_card_clients_default: BILL_AMT1 found cat
# customer_satisfaction_in_airline: Several missed categorical columns (likely low cardinality), Age found as cat
# Food_Delivery_Time: Type_of_order and Type_of_Vehicle found as cat
# heloc: One categorical found
# houses: Age found to be cat
# HR_Analytics_Job_Change_of_Data_Scientists: city_development_index found as cat (interaction test does not yield results, as there are almost no other numerical features)
# kddcup09_appetency: some false discoveries & missed ones
# (!) maternal_health_risk: Several categoricals although all are numerical - multiclass might be an issue!
# online_shoppers_intention: One missed categorical, several found, Region found to be irrelevant
# physiochemical_protein: Several false categoricals
# APSFailure: Several false categoricals
# Bioresponse: Several false categoricals
# qsar-biodeg: One missed, one false categorical
# students_dropout_and_academic_success: Several false categoricals
# MIC: Several missed
# superconductivity: Several (false?) categoricals

In [ ]:
cv_func = make_cv_scores_with_early_stopping(target_type=target_type,)
# Index(['BlastFurnaceSlag', 'Water', 'CoarseAggregate', 'FineAggregate', 'Age'], dtype='object')  
col = X.columns[0]

X_num = X.astype(float)

for ncol, col in enumerate(['RESOURCE']): # enumerate(X.columns):
    print(col)
    interaction_performances = {}
    interaction_significances = {}

    ### 1. Get Interactions
    cols_use = X_num.drop(col,axis=1).columns
    f_0 = X_num[col]

    features = {}
    for num, col2 in enumerate(cols_use):
        features[f"{col}_/_{col2}"] = X_num[col] / X_num[col2].replace(0, np.nan)
        features[f"{col}_x_{col2}"] = X_num[[col, col2]].prod(axis=1)
        features[f"{col}_-_{col2}"] = X_num[col].astype(np.float32) - X_num[col2].astype(np.float32)
        features[f"{col}_+_{col2}"] = X_num[[col,col2]].sum(axis=1)

    X_int = pd.DataFrame(features)

    ### 2. Get Highest Correlation
    corr_X_int = X_int.corrwith(y, method='spearman').abs().sort_values(ascending=False)

    # Basic method
    #highest_corr = X_int.corrwith(y, method='spearman').abs().sort_values(ascending=False).index[0]

    # base_cors = X_num.corrwith(y).abs().sort_values(ascending=False)
    # int_cors = X_int.corrwith(y, method='spearman').abs().to_frame()
    # int_cors_diff = int_cors.apply(lambda x: float([x-base_cors.loc[x.name.split(f"_{i}_")].max() for i in ['+', '-', '/', 'x'] if len(x.name.split(f"_{i}_"))>1][0][0]),axis=1)
    # highest_corr = int_cors_diff.sort_values(ascending=False).index[2]

    for highest_corr in corr_X_int.index:

        col2 = [highest_corr.split(f"_{i}_")[1] for i in ['+', '-', '/', 'x'] if len(highest_corr.split(f"_{i}_"))>1][0]
        x = X_int[highest_corr]
        base_performance = pd.Series({col: np.mean(detector.scores[col]['mean']) for col in [col,col2]})
        stronger_col = base_performance.idxmax()
        print(base_performance)

        ### 3. Get regular, binned and polynomial performance of the interaction feature
        # Regular TE performance
        if target_type == "regression":
            interaction_performances['mean'] = cv_func(x, y, Pipeline(steps=[('model', TargetMeanRegressor())]))
        else:
            interaction_performances['mean'] = cv_func(x, y, Pipeline(steps=[('model', TargetMeanClassifier())]))
        print(f"{x.name}: {np.mean(interaction_performances['mean']).round(2)}")

        # # Binned performance
        for bins in [16, 32, 64, 128, 256]:
            pipe = Pipeline([
                ("bin", LightGBMBinner(bins)),
                ("model", TargetMeanRegressor()) if target_type == "regression" else ("model", TargetMeanClassifier())
            ])
            interaction_performances[f'binned{bins}'] = cv_func(x.to_frame(), y, pipe)
            print(f"{x.name}-binned{bins}: {np.mean(interaction_performances[f'binned{bins}']).round(2)}")

            interaction_significances[f'interaction_binned{bins}_over_basecols'] = p_value_wilcoxon_greater_than_zero(
                                    interaction_performances[f'binned{bins}'] - detector.scores[stronger_col]['mean']
            )


        # Polynomial performance
        pipe = Pipeline([
        ("standardize", QuantileTransformer(n_quantiles=np.min([x.nunique(), 1000]), random_state=42)),
        ("impute", SimpleImputer(strategy="median")),
        # ("standardize", StandardScaler()),
        ("model", PolynomialRegressor(3)) if target_type == "regression" else ("model", PolynomialLogisticClassifier(3))
            ])
        interaction_performances['poly3'] = cv_func(x.to_frame(), y, pipe)
        print(f"{x.name}-poly3: {np.mean(interaction_performances['poly3']).round(2)}")


        ### 4. Test whether the interaction improves over the original features and whether it itself is numeric
        interaction_significances[f'interaction_mean_over_basecols'] = p_value_wilcoxon_greater_than_zero(
                                interaction_performances['mean'] - detector.scores[stronger_col]['mean']
        )


        interaction_significances[f'interaction_poly3_over_basecols'] = p_value_wilcoxon_greater_than_zero(
                                interaction_performances['poly3'] - detector.scores[stronger_col]['mean']
        )

        ### 5. Test whether categorical interaction is better than numerical interaction
        # Combine interaction performance
        x = X[col].astype(str) + X_num[col2].astype(str)
        if target_type == "regression":
            interaction_performances['combine'] = cv_func(x, y, Pipeline(steps=[('model', TargetMeanRegressor())]))
        else:
            interaction_performances['combine'] = cv_func(x, y, Pipeline(steps=[('model', TargetMeanClassifier())]))
        print(f"{col}&{col2}: {np.mean(interaction_performances['combine']).round(2)}")

        # # 5. Get groupby performance

        for stat in ['mean']: #, 'std', 'min', 'max', 'median']:
            if stat == 'mean':
                value_map = X_num[col2].groupby(X[col]).mean()
            elif stat == 'std':
                value_map = X_num[col2].groupby(X[col]).std()
            elif stat == 'min':
                value_map = X_num[col2].groupby(X[col]).min()
            elif stat == 'max':
                value_map = X_num[col2].groupby(X[col]).max()
            elif stat == 'median':
                value_map = X_num[col2].groupby(X[col]).median()

            x = X[col].map(value_map)
            if target_type == "regression":
                interaction_performances[f'{col2}By{col}-{stat}'] = cv_func(x, y, Pipeline(steps=[('model', TargetMeanRegressor())]))
            else:
                interaction_performances[f'{col2}By{col}-{stat}'] = cv_func(x, y, Pipeline(steps=[('model', TargetMeanClassifier())]))
            print(f"{col2}By{col}-{stat}: {np.mean(interaction_performances[f'{col2}By{col}-{stat}']).round(2)}")

            # # Binned performance
            for bins in [16, 32, 64, 128, 256]:
                pipe = Pipeline([
                    ("bin", LightGBMBinner(bins)),
                    ("model", TargetMeanRegressor()) if target_type == "regression" else ("model", TargetMeanClassifier())
                ])
                interaction_performances[f'{col2}By{col}-{stat}-binned{bins}'] = cv_func(x.to_frame(), y, pipe)
                print(f"{col2}By{col}-{stat}-binned{bins}: {np.mean(interaction_performances[f'{col2}By{col}-{stat}-binned{bins}']).round(2)}")

            # Polynomial performance
            # pipe = Pipeline([
            # ("standardize", QuantileTransformer(n_quantiles=np.min([x.nunique(), 1000]), random_state=42)),
            # ("impute", SimpleImputer(strategy="median")),
            # # ("standardize", StandardScaler()),
            # ("model", PolynomialRegressor(3)) if target_type == "regression" else ("model", PolynomialLogisticClassifier(3))
            #     ])
            # interaction_performances[f'{col2}By{col}-poly3'] = cv_func(x.to_frame(), y, pipe)
            # print(f"{col2}By{col}-poly3: {np.mean(interaction_performances[f'{col2}By{col}-poly3']).round(2)}")

            # for bins in [2, 4, 8, 16, 32, 64, 128]:

            #     dt = DecisionTreeRegressor(
            #         max_leaf_nodes = bins,     # desired number of bins
            #         min_samples_leaf = 1,   # min pts per bin to avoid overfitting
            #         # criterion = "entropy"   # or "gini"
            #     )
            #     X_int[highest_corr]

            #     print(f"Bins: {bins}, CV Score: {np.mean(cv_func(X_int[highest_corr].to_frame(), y, Pipeline(steps=[('model', dt)])))}")


    # if ncol==0:
    #     break

### TargetMean with cut stuff - probably already covered

In [ ]:
import openml
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import *
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
import pickle
import os

def drop_infrequent_values(series, thresh=1):
    value_counts = series.value_counts()
    non_unique_values = value_counts[value_counts > thresh].index
    return series[series.isin(non_unique_values)]

def drop_frequent_values(series, thresh=1):
    value_counts = series.value_counts()
    non_unique_values = value_counts[value_counts <= thresh].index
    return series[series.isin(non_unique_values)]

from utils import make_cv_scores_with_early_stopping, TargetMeanClassifier
cv = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores_with_early_stopping = make_cv_scores_with_early_stopping("binary", roc_auc_score, cv, early_stopping_rounds=20)

class TargetMeanClassifierCut(TargetMeanClassifier):
    def __init__(self, q_thresh=0):
        super().__init__()
        self.q_thresh = q_thresh

    def fit(self, X, y):
        X_use = X.copy()
        self.c_map = dict(X_use.value_counts())
        self.c_map = {k: k  if v > self.q_thresh else 'nan' for k, v in self.c_map.items()}  # only keep those with more than 5 occurrences
        X_use = X_use.map(self.c_map)
        return super().fit(X_use, y)

    def predict_proba(self, X):
        X_use = X.copy()
        X_use = X_use.map(self.c_map)
        return super().predict_proba(X_use)

class TargetMeanRegressorCut(TargetMeanRegressor):
    def __init__(self, q_thresh=0):
        super().__init__()
        self.q_thresh = q_thresh

    def fit(self, X, y):
        # TODO: Make sure that the category dtype of train and val matches
        X_use = X.copy()
        self.c_map = dict(X_use.value_counts())
        self.c_map = {k: k  if v > self.q_thresh else 'nan' for k, v in self.c_map.items()}  # only keep those with more than 5 occurrences
        X_use = X_use.map(self.c_map)
        return super().fit(X_use, y)
    
    def predict(self, X):
        X_use = X.copy()
        X_use = X_use.map(self.c_map)
        return super().predict(X_use)


from lightgbm import LGBMClassifier, LGBMRegressor
class LightGBMClassifierCut(LGBMClassifier):
    def __init__(self, q_thresh=0, init_kwargs: dict=dict()):
        super().__init__(**init_kwargs)
        self.verbose = init_kwargs.get("verbose", -1)
        self.n_estimators = init_kwargs.get("n_estimators", 100)
        self.max_bin = init_kwargs.get("max_bin", 255)
        self.max_depth = init_kwargs.get("max_depth", 2)
        self.random_state = init_kwargs.get("random_state", 42)
        self.init_kwargs = None
        self.q_thresh = q_thresh

    def fit(self, X, y, **kwargs):
        X_use = X.iloc[:,0].copy()
        if X_use.dtype in ['object', 'category']:
            fill = 'nan'
        else:
            fill = np.nan
        self.c_map = dict(X_use.value_counts())
        self.c_map = {k: k  if v > self.q_thresh else fill for k, v in self.c_map.items()}  # only keep those with more than 5 occurrences
        X_use = X_use.map(self.c_map)
        if X_use.dtype in ['object', 'category']:
            X_use = X_use.astype('category')
            self.dt = X_use.dtype 
        return super().fit(X_use.to_frame(), y, **kwargs)
    
    def predict_proba(self, X, **kwargs):
        X_use = X.iloc[:,0].copy()
        X_use = X_use.map(self.c_map)
        if X_use.dtype in ['object', 'category']:   
            X_use = X_use.astype(self.dt)
        return super().predict_proba(X_use.to_frame(), **kwargs)

class LightGBMRegressorCut(LGBMRegressor):
    def __init__(self, q_thresh=0, init_kwargs: dict=dict()):
        super().__init__(**init_kwargs)
        self.verbose = init_kwargs.get("verbose", -1)
        self.n_estimators = init_kwargs.get("n_estimators", 100)
        self.max_bin = init_kwargs.get("max_bin", 255)
        self.max_depth = init_kwargs.get("max_depth", 2)
        self.random_state = init_kwargs.get("random_state", 42)
        self.init_kwargs = None
        self.q_thresh = q_thresh

    def fit(self, X, y, **kwargs):
        X_use = X.iloc[:,0].copy()
        if X_use.dtype in ['object', 'category']:
            fill = 'nan'
        else:
            fill = np.nan
        self.c_map = dict(X_use.value_counts())
        self.c_map = {k: k  if v > self.q_thresh else fill for k, v in self.c_map.items()}  # only keep those with more than 5 occurrences
        X_use = X_use.map(self.c_map)
        if X_use.dtype in ['object', 'category']:
            X_use = X_use.astype('category')
            self.dt = X_use.dtype 
        return super().fit(X_use.to_frame(), y, **kwargs)

    def predict(self, X, **kwargs):
        X_use = X.iloc[:,0].copy()
        X_use = X_use.map(self.c_map)
        if X_use.dtype in ['object', 'category']:   
            X_use = X_use.astype(self.dt)
        return super().predict(X_use.to_frame(), **kwargs)
    
def safe_stratified_group_kfold(X, y, groups, n_splits=5, max_attempts=1):
    sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)
    attempt = 0

    while attempt < max_attempts:
        for fold, (train_idx, test_idx) in enumerate(sgkf.split(X, y, groups)):
            test_labels = y[test_idx]
            class_counts = Counter(test_labels)
            if len(class_counts) < 2:
                break  # This fold has only one class
        else:
            return sgkf#.split(X, y, groups)  # All folds are good
        
        attempt += 1

    print("Could not generate stratified group folds with both classes in all test sets.")
    return None  # If we reach here, it means we couldn't find a valid split

def grouped_interpolation_test(x,y, target_type, add_dummy=False):
    q = int(x.nunique())
    
    if target_type == 'binary':
        # cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
        cv = safe_stratified_group_kfold(x, y, x, n_splits=5)
        if cv is None:
            return None
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "binary", roc_auc_score, cv, early_stopping_rounds=20, 
            verbose=False, groups=x
            )

        lgb_model = LGBMClassifier(verbose=-1, n_estimators=q, random_state=42, max_bin=q, max_depth=2)
    elif target_type == 'regression':
        cv = GroupKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "regression", root_mean_squared_error, cv, early_stopping_rounds=20, 
            verbose=False, groups=x
            )

        lgb_model = LGBMRegressor(verbose=-1, n_estimators=q, random_state=42, max_bin=q, max_depth=2)


    if add_dummy:
        pipe = Pipeline([("model", DummyRegressor())])
        res = cv_scores_with_early_stopping(X, y, pipe)
        print(f"Dummy: {col}: {np.mean(res):.4f} (+/- {np.std(res):.4f})")    

    pipe = Pipeline([("model", lgb_model)])
    res = cv_scores_with_early_stopping(X[col].astype(float).to_frame(), y, pipe)
    print(f"{col}: {np.mean(res):.4f} (+/- {np.std(res):.4f})")    
    return res


def analyze_cat_feature(x,y):
    # Insight: histograms look different depending on whether we treat a feature as string or float - could somehow use this information
    fig,ax = plt.subplots(1,4, figsize=(10, 5))
    ax[0].scatter(x, y)

    y_by_x = y.groupby(x, observed=False).mean()
    ax[1].scatter(y_by_x.index, y_by_x.values)

    ax[2].hist(x, bins=100)
    pd.Series(x.astype(float).sort_values()).plot(kind='hist', bins=100, ax=ax[3])

def get_feature_stats(x, y, target_type='binary', verbose=True):
    q = int(x.nunique())
    lgb_base_params = {
        "verbose": -1, "n_estimators": q*10, "random_state": 42, "max_bin": q, 
        "max_depth": 2, "min_samples_leaf": 1, "min_child_samples": 1,
    }
    stats = {}
    stats['q'] = q
    if target_type == 'binary':
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "binary", roc_auc_score, cv, early_stopping_rounds=20, 
            verbose=False
            )

        lgb_class = lambda lgb_params: LGBMClassifier(**lgb_params)
        target_model = TargetMeanClassifier()
        target_cut_model = lambda t: TargetMeanClassifierCut(q_thresh=t)
        lgb_cut_model = lambda t, p: LightGBMClassifierCut(q_thresh=t, init_kwargs=p)
    elif target_type == 'regression':
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "regression", root_mean_squared_error, cv, early_stopping_rounds=20, 
            verbose=False
            )

        lgb_class = lambda lgb_params: LGBMRegressor(**lgb_params)
        target_model = TargetMeanRegressor()
        target_cut_model = lambda t: TargetMeanRegressorCut(q_thresh=t)
        lgb_cut_model = lambda t, p: LightGBMRegressorCut(q_thresh=t, init_kwargs=p)
    else:
        raise ValueError("Unsupported target type. Use 'binary' or 'regression'.")


    infreq = x.value_counts().sort_values(ascending=True).values[:5]

    # Target-based stats
    pipe = Pipeline([("model", target_model)])
    stats['mean'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    for t in infreq:
        t = int(t)

        pipe = Pipeline([("model", target_cut_model(t))])
        stats[f'mean-u>{t}'] = cv_scores_with_early_stopping(x.astype('category'), y, pipe)

        pipe = Pipeline([("model", target_model)])
        x_use = drop_infrequent_values(x,t)
        y_use = y.loc[x_use.index]
        stats[f'mean-drop-u<={t}'] = cv_scores_with_early_stopping(x_use.astype('category'), y_use, pipe)

        try:
            pipe = Pipeline([("model", target_model)])
            x_use = drop_frequent_values(x,t)
            y_use = y.loc[x_use.index]
            stats[f'mean-drop-u>{t}'] = cv_scores_with_early_stopping(x_use.astype('category'), y_use, pipe)
        except:
            continue
    # LGB-based stats
    pipe = Pipeline([("model", lgb_class(lgb_base_params))])
    stats['lgb-num'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)
    pipe = Pipeline([("model", lgb_class(lgb_base_params))])
    stats['lgb-cat'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    for t in infreq:
        t = int(t)
        pipe = Pipeline([("model", lgb_cut_model(t, lgb_base_params))])
        stats[f'lgb-num-u>{t}'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)

        pipe = Pipeline([("model", lgb_cut_model(t, lgb_base_params))])
        stats[f'lgb-cat-u>{t}'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)

    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=1, init_kwargs=lgb_base_params))])
    # stats['lgb-num-u>1'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=2, init_kwargs=lgb_base_params))])
    # stats['lgb-num-u>2'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)

    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=1, init_kwargs=lgb_base_params))])
    # stats['lgb-cat-u>1'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=2, init_kwargs=lgb_base_params))])
    # stats['lgb-cat-u>2'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)

    # for n_est in [int(q/2),int(q/4), int(q/8)]:
    #     p = lgb_base_params.copy()
    #     p["n_estimators"] = n_est
    #     pipe = Pipeline([("model", lgb_class(p))])
    #     stats[f'lgb-{n_est}est-num'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)
    #     pipe = Pipeline([("model", lgb_class(p))])
    #     stats[f'lgb-{n_est}est-cat'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)

    if q>16:
        for m_bin in [int(q/2),int(q/4), int(q/8), int(q/16)]:
            # Cat features not affected 
            p = lgb_base_params.copy()
            p["max_bin"] = m_bin
            pipe = Pipeline([("model", lgb_class(p))])
            stats[f'lgb-{m_bin}bins-num'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)


    # p = lgb_base_params.copy()
    # p["max_depth"] = 1
    # pipe = Pipeline([("model", lgb_class(p))])
    # stats['lgb-d1-cat'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=1, init_kwargs=p))])
    # stats['lgb-d1-cat-u>1'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=2, init_kwargs=p))])
    # stats['lgb-d1-cat-u>2'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)


    if verbose:
        print(f"Feature: {x.name}")
        for key, value in stats.items():
            if key in ['q']:
                print(f"{key}: {value}")
            else:
                print(f"{key}: {np.mean(value):.4f} (+/- {np.std(value):.4f})")

    return stats

In [ ]:
def get_optimal_cat_thresh(x, y, target_type='binary', verbose=True):
    q = int(x.nunique())
    lgb_base_params = {
        "verbose": -1, "n_estimators": q*10, "random_state": 42, "max_bin": q, 
        "max_depth": 2, "min_samples_leaf": 1, "min_child_samples": 1,
    }
    stats = {}
    # stats['q'] = q
    if target_type == 'binary':
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "binary", roc_auc_score, cv, early_stopping_rounds=20, 
            verbose=False
            )

        lgb_class = lambda lgb_params: LGBMClassifier(**lgb_params)
        target_model = TargetMeanClassifier()
        target_cut_model = lambda t: TargetMeanClassifierCut(q_thresh=t)
        lgb_cut_model = lambda t, p: LightGBMClassifierCut(q_thresh=t, init_kwargs=p)
    elif target_type == 'regression':
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "regression", root_mean_squared_error, cv, early_stopping_rounds=20, 
            verbose=False
            )

        lgb_class = lambda lgb_params: LGBMRegressor(**lgb_params)
        target_model = TargetMeanRegressor()
        target_cut_model = lambda t: TargetMeanRegressorCut(q_thresh=t)
        lgb_cut_model = lambda t, p: LightGBMRegressorCut(q_thresh=t, init_kwargs=p)
    else:
        raise ValueError("Unsupported target type. Use 'binary' or 'regression'.")


    infreq = x.value_counts().sort_values(ascending=True).unique()[:100]

    # Target-based stats
    pipe = Pipeline([("model", target_model)])
    stats['mean'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    for t in infreq:
        t = int(t)

        pipe = Pipeline([("model", target_cut_model(t))])
        stats[f'mean-u>{t}'] = cv_scores_with_early_stopping(x.astype('category'), y, pipe)

        # pipe = Pipeline([("model", target_model)])
        # x_use = drop_infrequent_values(x,t)
        # y_use = y.loc[x_use.index]
        # stats[f'mean-drop-u<={t}'] = cv_scores_with_early_stopping(x_use.astype('category'), y_use, pipe)

        # try:
        #     pipe = Pipeline([("model", target_model)])
        #     x_use = drop_frequent_values(x,t)
        #     y_use = y.loc[x_use.index]
        #     stats[f'mean-drop-u>{t}'] = cv_scores_with_early_stopping(x_use.astype('category'), y_use, pipe)
        # except:
        #     continue

    if verbose:
        print(f"Feature: {x.name}")
        for key, value in stats.items():
            if key in ['q']:
                print(f"{key}: {value}")
            else:
                print(f"{key}: {np.mean(value):.4f} (+/- {np.std(value):.4f})")

    plt.plot(pd.DataFrame(stats).mean(axis=0))
    plt.title(f"Feature: {x.name}")
    plt.show()

    return stats

In [ ]:
def get_feature_stats(x, y, target_type='binary', verbose=True):
    q = int(x.nunique())
    lgb_base_params = {
        "verbose": -1, "n_estimators": q*10, "random_state": 42, "max_bin": q, 
        "max_depth": 2, "min_samples_leaf": 1, "min_child_samples": 1,
    }
    stats = {}
    stats['q'] = q
    if target_type == 'binary':
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "binary", roc_auc_score, cv, early_stopping_rounds=20, 
            verbose=False
            )

        lgb_class = lambda lgb_params: LGBMClassifier(**lgb_params)
        target_model = TargetMeanClassifier()
        target_cut_model = lambda t: TargetMeanClassifierCut(q_thresh=t)
    elif target_type == 'regression':
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores_with_early_stopping = make_cv_scores_with_early_stopping(
            "regression", root_mean_squared_error, cv, early_stopping_rounds=20, 
            verbose=False
            )

        lgb_class = lambda lgb_params: LGBMRegressor(**lgb_params)
        target_model = TargetMeanRegressor()
        target_cut_model = lambda t: TargetMeanRegressorCut(q_thresh=t)
    else:
        raise ValueError("Unsupported target type. Use 'binary' or 'regression'.")

    # Target-based stats
    pipe = Pipeline([("model", target_model)])
    stats['mean'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    for t in range(1,5):
        pipe = Pipeline([("model", target_cut_model(t))])
        stats[f'mean-u>{t}'] = cv_scores_with_early_stopping(x.astype('category'), y, pipe)

    for t in range(1,5):
        pipe = Pipeline([("model", target_model)])
        x_use = drop_infrequent_values(x,t)
        y_use = y.loc[x_use.index]
        stats[f'mean-drop-l<={t}'] = cv_scores_with_early_stopping(x_use.astype('category'), y_use, pipe)
    for t in range(1,5):
        pipe = Pipeline([("model", target_model)])
        x_use = drop_frequent_values(x,t)
        y_use = y.loc[x_use.index]
        stats[f'mean-drop-l>{t}'] = cv_scores_with_early_stopping(x_use.astype('category'), y_use, pipe)

    # LGB-based stats
    pipe = Pipeline([("model", lgb_class(lgb_base_params))])
    stats['lgb-num'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)
    pipe = Pipeline([("model", lgb_class(lgb_base_params))])
    stats['lgb-cat'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    for t in range(5):
        pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=t, init_kwargs=lgb_base_params))])
        stats[f'lgb-num-u>{t}'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)

        pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=t, init_kwargs=lgb_base_params))])
        stats[f'lgb-cat-u>{t}'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)

    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=1, init_kwargs=lgb_base_params))])
    # stats['lgb-num-u>1'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=2, init_kwargs=lgb_base_params))])
    # stats['lgb-num-u>2'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)

    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=1, init_kwargs=lgb_base_params))])
    # stats['lgb-cat-u>1'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=2, init_kwargs=lgb_base_params))])
    # stats['lgb-cat-u>2'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)

    # for n_est in [int(q/2),int(q/4), int(q/8)]:
    #     p = lgb_base_params.copy()
    #     p["n_estimators"] = n_est
    #     pipe = Pipeline([("model", lgb_class(p))])
    #     stats[f'lgb-{n_est}est-num'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)
    #     pipe = Pipeline([("model", lgb_class(p))])
    #     stats[f'lgb-{n_est}est-cat'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)


    for m_bin in [int(q/2),int(q/4), int(q/8), int(q/16)]:
        # Cat features not affected 
        p = lgb_base_params.copy()
        p["max_bin"] = m_bin
        pipe = Pipeline([("model", lgb_class(p))])
        stats[f'lgb-{m_bin}bins-num'] = cv_scores_with_early_stopping(x.astype(float).to_frame(), y, pipe)


    # p = lgb_base_params.copy()
    # p["max_depth"] = 1
    # pipe = Pipeline([("model", lgb_class(p))])
    # stats['lgb-d1-cat'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=1, init_kwargs=p))])
    # stats['lgb-d1-cat-u>1'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)
    # pipe = Pipeline([("model", LightGBMClassifierCut(q_thresh=2, init_kwargs=p))])
    # stats['lgb-d1-cat-u>2'] = cv_scores_with_early_stopping(x.astype('category').to_frame(), y, pipe)


    if verbose:
        print(f"Feature: {x.name}")
        for key, value in stats.items():
            if key in ['q']:
                print(f"{key}: {value}")
            else:
                print(f"{key}: {np.mean(value):.4f} (+/- {np.std(value):.4f})")

    # return stats

get_feature_stats(X['RESOURCE'], y)

### Remaining issues
- Still false positives on datasets with many features like 
    - APSFailure: ['ae_000', 'af_000', 'ak_000', 'ar_000', 'as_000', 'au_000', 'cf_000', 'cz_000', 'da_000', 'db_000', 'df_000', 'dh_000', 'dj_000', 'dk_000', 'dl_000', 'dr_000', 'dy_000', 'dz_000', 'ea_000', 'ef_000', 'eg_000']
    - Bioresponse: ['D2', 'D14', 'D31', 'D32', 'D33', 'D35', 'D36', 'D38', 'D39', 'D40', 'D44', 'D45', 'D46', 'D48', 'D49', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D60', 'D61', 'D62', 'D63', 'D64', 'D65', 'D67', 'D68', 'D69', 'D71', 'D74', 'D75', 'D76', 'D87', 'D95', 'D103', 'D547', 'D754', 'D771', 'D866']
- Ordinals like age often found to be categorical 
    - (bank & house datasets)
    - Bank_Customer_Churn: age
    - airline satisfaction: Age
    - miami_housing: age


- On churn we find state to be numeric (which might be a good thing)
- On MarketingCampaign some new categoricals are only identified as such with AG preprocessing: ['Year_Birth', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Dt_Customer.month']
- Several other FPs (likely mostly ordinals)
    - coil: ['contributionFirePolicies', 'numberOfFirePolicies']
    - concrete: ['BlastFurnaceSlag', 'Water', 'CoarseAggregate', 'FineAggregate', 'Age']
    - credit default: ['BILL_AMT1']
    - maternal_health_risk: ['Age', 'SystolicBP', 'BS', 'BodyTemp', 'HeartRate']
    - online_shoppers_intention: ['Informational', 'SpecialDay']
    - students dropout: ['Previous_qualification_grade', 'Curricular_units_1st_sem_enrolled', 'Curricular_units_1st_sem_evaluations', 'Curricular_units_2nd_sem_enrolled', 'Curricular_units_2nd_sem_approved']
    - MIC: ['NA_BLOOD']
- Several FPs on the special chemistry datasets
    - physio: ['TotalSurfaceArea', 'NonPolarExposedArea', 'MassWeightedExposedArea', 'EuclideanDistance']
    - qsar-biodeg: ['Laplace_MoharIndex2']
    - superconduct: almost all
- AG does quite often find new numerics!!



Positive:
- Almost no new numeric!
    - state in churn (good thing likely)
    - airline: ['DepartureArrivaltimeconvenient', 'Foodanddrink', 'Inflightwifiservice', 'Inflightentertainment', 'EaseofOnlinebooking', 'Onboardservice', 'Checkinservice', 'Cleanliness']
    - online shoppers intention: ['OperatingSystems']
    - qsar-biodeg: ['C026_chemical_substructure']
- Many clear numerics are solved


Possible additional tests:
- Interaction test
    - Check whether features interact linearly with other features, which is another sign of being numerical

- LOO vs. univariate linear regression - if LOO performs better, we have a clear pattern.
    - Would certainly rule out several features immediately that we havent covered yet, i.e. in anneal dataset. Also one uninformative featue in Amazon dataset

- Unique removal test
    - Do something about unique values - i.e. a test that finds whether we can infer information for them by using the feature as numeric
- stratified CV using the feature as target: 
    - Perfect condition for mean scenario, if it still can be beaten, we likely have a numeric feature

Tried: 
- performance test: Rule out the features interacting with the target in a complex way using LGB, as previously
    - Interestingly is always the same as after the previous tests - it is impossible to find something new